In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
var('xx')
Q.<i> = NumberField(xx^2+1)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [4]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [5]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
line = Line([y, z])

In [7]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [162]:
general_cubic

x^2*z*b*c + x*y*t*b*c - x*z*t*b*c - y*t^2*b*c - x*z^2*c^2 - x*z*t*c^2 + y*z*t*c^2 + y*t^2*c^2 + x*y*z*c*d - x*z^2*c*d + y^2*t*c*d - y*z*t*c*d - x^2*z*c*e + x*y*z*c*e + x*y*t*c*e - y^2*t*c*e - x^2*z*c*f + x*y*z*c*f + x*y*t*c*f - y^2*t*c*f + 2*x^2*y*e*f - 2*x*y^2*e*f + x*z^2*e*f - x*z*t*e*f + y*z*t*e*f - y*t^2*e*f

In [8]:
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [9]:
%time adm_SE1 = SE1.find_admissible_projectivities()

CPU times: user 12.5 s, sys: 274 ms, total: 12.7 s
Wall time: 20.1 s


In [10]:
len(adm_SE1)

576

In [11]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)

CPU times: user 8.89 s, sys: 385 ms, total: 9.27 s
Wall time: 27.1 s


In [12]:
len(simm_SE1)

2

In [13]:
adm_perm = SE1.find_admissible_permutations()
keys = list(SE1.cl_lines.keys())

In [14]:
len(adm_perm)

1152

In [22]:
L_sets = [['E1', 'G4', 'E2', 'G3', 'E3'], ['E1', 'G4', 'E2', 'F12', 'E3'],
          ['E1', 'G4', 'E2', 'G5', 'F34'], ['E1', 'G4', 'E2', 'G6', 'F34'], 
          ['E1', 'G4', 'F24', 'G2', 'F34'], ['E1', 'G4', 'F24', 'F13', 'F34'], 
          ['F14', 'G4', 'F24', 'F35', 'F34'], ['F14', 'G4', 'F24', 'F36', 'F34'], 
          ['E1', 'G4', 'E3', 'G2', 'E2'], ['E1', 'G4', 'E3', 'F13', 'E2'], 
          ['F14', 'G4', 'E3', 'F35', 'E2'], ['F14', 'G4', 'E3', 'F36', 'E2']]

In [23]:
len(L_sets)

12

In [24]:
M12 = SE1.find_all_proj_parallel(L_sets)

In [28]:
len(M12)

12

In [29]:
SE1.find_simmetries(M12)

[
[1 0 0 0]  [                        4*c^2                             0                             0 2*c^2 - 2*c*e - 2*c*f + 2*e*f]
[0 1 0 0]  [                            0                         4*c^2                             0        -2*c*d + 2*c*e + 2*c*f]
[0 0 1 0]  [                            0                             0                         4*c^2        -2*c^2 + 2*c*d - 2*e*f]
[0 0 0 1], [                            0                             0                             0                        -4*c^2]
]

In [32]:
SE1.find_conditions_for_subfamilies(M12, simm_SE1)

[]

In [ ]:
#verificare che se una delle matrici fissa cubica liscia, cubica ha punto di eckardt

In [33]:
M2 = [mat for mat in simm_SE1 if mat != matrix.identity(4)][0]
M2_perm = Permutation(apply_proj_to_lines(M2, list(SE1.cl_lines.values()))).to_permutation_group_element()
print(M2_perm)

(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23)


In [34]:
perms_lines = []
for perm in adm_perm:
    perms_lines.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())

In [35]:
Gr = PermutationGroup(perms_lines)

In [36]:
Gr.order()

1152

In [37]:
M2_perm.order()

2

In [ ]:
for el in Gr:
    print(el.order())

In [38]:
el_order_4 = []
for el in Gr.list():
    iden = Gr.identity()
    if el^2 == M2_perm and el^3 != iden and el^4 == iden:
        el_order_4.append(el)

In [39]:
len(el_order_4)

12

In [71]:
labels_el_order_4 = [from_perm_to_labels(el) for el in el_order_4]
for labels in labels_el_order_4:
    print([labels[keys.index(label)] for label in ['E1', 'G4', 'E2', 'G3', 'E3', 'E5']])

['E1', 'G4', 'E5', 'F15', 'E6', 'F24']
['E1', 'G4', 'F45', 'G5', 'F46', 'E2']
['E1', 'G4', 'E6', 'F16', 'E5', 'F34']
['E1', 'G4', 'F46', 'G6', 'F45', 'E3']
['E1', 'G4', 'E3', 'G6', 'F24', 'F46']
['E1', 'G4', 'F34', 'F16', 'E2', 'E6']
['E1', 'G4', 'E5', 'G2', 'F46', 'F24']
['E1', 'G4', 'F45', 'F12', 'E6', 'E2']
['E1', 'G4', 'E3', 'G5', 'F24', 'E6']
['E1', 'G4', 'F34', 'F15', 'E2', 'F46']
['E1', 'G4', 'E6', 'G2', 'F45', 'E3']
['E1', 'G4', 'F46', 'F12', 'E5', 'F34']


In [41]:
projs = SE1.find_admissible_projectivities(labels_el_order_4)

In [73]:
len(projs)

12

In [152]:
mins = list(set(matrix([SE1.cl_lines['G3'].apply_proj(projs[1]).plucker, SE1.cl_lines['G5'].plucker]).minors(2)))

In [150]:
mins = [remove_sing_factors(el, SE1.sing_cubic) for el in mins if el !=0]

In [154]:
[el.factor() for el in mins if el != 0]

[]

In [143]:
P.ideal(mins).groebner_basis()

Polynomial Sequence with 72 Polynomials in 4 Variables

In [97]:
def find_conditions_for_subfamilies(cubic, projectivities, simmetries):
    mon = ((x+y+z+t)^3).monomials()
    conditions = []
    for M in [proj for proj in projectivities if proj not in simmetries]:
        print(projectivities.index(M))
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), cubic.sing_cubic)    
        minor = list(set(matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)))
        minor = [remove_sing_factors(el, cubic.sing_cubic) for el in minor if el !=0]
        ideale = cubic.P.ideal(minor)
        print(ideale)
        new_ideals = get_valid_ideals(ideale, cubic)
        for ideal in new_ideals:            
            prim_deco2 = (ideale + ideal).radical().primary_decomposition()
            for ideal2 in prim_deco2:
                if is_ideal_valid(cubic, ideal2):
                    conditions.append(ideal2.gens())                        
    return list(set(conditions))


def get_valid_ideals(ideale, cubic):
    valid_ideals = []
    vrs = [c,d,e,f]
    for vr in vrs:
        valid_ideals_var = []
        del_ideale = ideale.elimination_ideal([vr])
        prim_deco_del = del_ideale.radical().primary_decomposition()
        for ideal in prim_deco_del:
            if is_ideal_valid(cubic, ideal):
                valid_ideals_var.append(ideal)
        if valid_ideals_var == []:
            return []
        else:
            valid_ideals.append(valid_ideals_var)
    return [id1+id2+id3+id4 for id1 in valid_ideals[0] for id2 in valid_ideals[1] for id3 in valid_ideals[2] for id4 in valid_ideals[3]] 

def is_ideal_valid(cubic, ideal):
    if cubic.sing_cubic.value() in ideal:
        return False
    for poly in list(set([pl.conditions for pl in cubic.tritangent_planes if pl.conditions != 0])):
        if poly in ideal:
            return False
    return True

In [ ]:
find_conditions_for_subfamilies(SE1, [projs[1]], simm_SE1)

In [81]:
Ide = Ideal(327*d*e + (6*i - 129)*e^2 + (14*i + 26)*c*f + (-48*i - 167)*d*f + (-56*i - 104)*e*f + (32*i + 75)*f^2, 654*c*e + (-504*i + 372)*e^2 + (23*i - 331)*c*f + (108*i + 76)*d*f + (562*i + 16)*e*f + (-181*i - 87)*f^2, 654*d^2 + (-24*i - 138)*e^2 + (-56*i - 104)*c*f + (192*i - 313)*d*f + (224*i + 89)*e*f + (-128*i + 27)*f^2, 654*c*d + (-192*i + 204)*e^2 + (-121*i - 69)*c*f + (-208*i - 106)*d*f + (-170*i - 378)*e*f + (175*i + 107)*f^2, 654*c^2 + (600*i + 180)*e^2 + (-17*i + 311)*c*f + (-4*i - 132)*d*f + (-586*i - 590)*e*f + (39*i + 197)*f^2, 1962*e^3 + (219*i - 21549)*e^2*f + (511*i + 949)*c*f^2 + (-1752*i - 2008)*d*f^2 + (-2044*i + 7322)*e*f^2 + (1168*i + 1266)*f^3)

In [96]:
Ide.elimination_ideal([e,f]).gens()[0]

2*c^5 + (-8*i + 6)*c^4*d + (-14*i - 3)*c^3*d^2 + (-8*i - 3)*c^2*d^3 + (-7*i - 11)*c*d^4 + (6*i - 6)*d^5

In [55]:
%time conds = find_conditions_for_subfamilies(SE1, projs, simm_SE1)

0
1
2
3
4
5
6
7
8
9
10
11
CPU times: user 4min 26s, sys: 4.27 s, total: 4min 30s
Wall time: 4min 34s


In [56]:
conds_linear = [cond for cond in conds if [el.degree() for el in cond] == [1,1,1]]
len(conds_linear)

28

In [57]:
non_linear_conds = [cond for cond in conds if cond not in conds_linear]
len(non_linear_conds)

60

In [ ]:
non_linear_conds

In [59]:
sost = solve_linear_system(conds_linear[0], [d,e,f], [c])
sost

[(10*i + 55)*c, 0, 25*c, (20*i - 15)*c]

In [60]:
var = [d,e,f,c]

In [61]:
nc = SE1.subs({var[i]:sost[i] for i in range(4)})
nc

(100*i + 550)*x^2*z + (1450*i - 1400)*x*y*z + (-350*i + 1200)*x*z^2 + (1100*i - 200)*x*y*t + (450*i - 650)*y^2*t + (-1550*i + 850)*y*z*t + (-1200*i - 350)*y*t^2

In [62]:
nc.eckardt_points

[(0, 0, 0, 1)]

In [63]:
%time nc_proj = nc.find_admissible_projectivities()

CPU times: user 2.61 s, sys: 301 ms, total: 2.91 s
Wall time: 10.1 s


In [64]:
len(nc_proj)

576

In [65]:
%time nc_simm = nc.find_simmetries(nc_proj)

CPU times: user 1.9 s, sys: 207 ms, total: 2.1 s
Wall time: 2.71 s


In [66]:
len(nc_simm)

4

In [39]:
for i in range(len(nc_simm)-1):
    mat1 = nc_simm[i]
    for j in range(i+1, len(nc_simm)):
        mat2 = nc_simm[j]
        are_matrices_equal(mat1, mat2)

In [67]:
Gnc = Group(nc_simm)

In [68]:
Gnc.get_order_of_elements()

[1, 4, 4, 2]

In [69]:
Gnc.is_group()

True

In [ ]:
%%time
for cond in conds_linear:
    sost = solve_linear_system(cond, [d,e,f], [c])
    nc = SE1.subs({var[i]:sost[i] for i in range(4)})
    print(len(nc.find_simmetries()))

In [45]:
with open('conditions.pickle', 'rb') as ff:
    conditions = pickle.load(ff)